# Convert And Inference Pytorch model with CustomOps

With onnxruntime_extensions package, the Pytorch model with the operation cannot be converted into the standard ONNX operators still be converted and the converted ONNX model still can be run with ONNXRuntime, plus onnxruntime_extensions package. This tutorial show it works

## Converting
Suppose there is a model which cannot be converted because there is no matrix inverse operation in ONNX standard opset. And the model will be defined like the following.

In [1]:
import torch
import torchvision

class CustomInverse(torch.nn.Module):
    def forward(self, x):
        return torch.inverse(x) + x

To export this model into ONNX format, we need register a custom op handler for pytorch.onn.exporter.

In [2]:
from torch.onnx import register_custom_op_symbolic


def my_inverse(g, self):
    return g.op("ai.onnx.contrib::Inverse", self)

register_custom_op_symbolic('::inverse', my_inverse, 1)

Then, invoke the exporter

In [3]:
import io
import onnx

x0 = torch.randn(3, 3)
# Export model to ONNX
f = io.BytesIO()
t_model = CustomInverse()
torch.onnx.export(t_model, (x0, ), f, opset_version=12)
onnx_model = onnx.load(io.BytesIO(f.getvalue()))

Now, we got a ONNX model in the memory, and it can be save into a disk file by 'onnx.save_model(onnx_model, <file_path>)

## Inference
This converted model cannot directly run the onnxruntime due to the custom operator. but it can run with onnxruntime_extensions easily.

Firstly, let define a PyOp function to inteprete the custom op node in the ONNNX model.

In [4]:
import numpy
from onnxruntime_extensions import onnx_op, PyOp
@onnx_op(op_type="Inverse")
def inverse(x):
    # the user custom op implementation here:
    return numpy.linalg.inv(x)


* **ONNX Inference**

In [5]:
from onnxruntime_extensions import PyOrtFunction
onnx_fn = PyOrtFunction.from_model(onnx_model)
y = onnx_fn(x0.numpy())
print(y)

[[-3.081008    0.20269153  0.42009977]
 [-3.3962293   2.5986686   2.4447646 ]
 [ 0.7805753  -0.20394287 -2.7528977 ]]


* **Compare the result with Pytorch**

In [7]:
t_y = t_model(x0)
numpy.testing.assert_almost_equal(t_y, y, decimal=5)

## Implement the customop in C++ (optional)
To make the ONNX model with the CustomOp runn on all other language supported by ONNX Runtime and be independdent of Python, a C++ implmentation is needed, check here for the [inverse.hpp](../operators/math/dlib/inverse.hpp) for an example on how to do that.

In [8]:
from onnxruntime_extensions import enable_py_op
# disable the PyOp function and run with the C++ function
enable_py_op(False)
y = onnx_fn(x0.numpy())
print(y)

[[-3.081008    0.20269153  0.42009977]
 [-3.3962293   2.5986686   2.4447646 ]
 [ 0.7805753  -0.20394287 -2.7528977 ]]
